In [6]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
# Import Dataset
df= pd.read_csv('/Users/vibhuverma/Desktop/CLASSWORK/BUSINESS PRACTICUM/Anti-Corruption/Data Provided/USAID_Project_Data/USAID_Anticorruption_Projects_Database.csv')
df.drop(['proj_res','rfp_rftop','final_res','mind_eval','final_eval','audit','interim_rep','addl_docs','addl_info'],axis=1,inplace=True)

In [9]:
# Convert to list
data = df.proj_desc.values.tolist()

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['The five components of this program seek to take what already exists and '
 'improve upon it, implement it and find ways to ensure that the progress '
 'achieved is sustainable over the long term. The project will help the '
 'government of the Islamic Republic of Afghanistan strengthen its High Office '
 'of Oversight (HOO) - making it a strong, effective institution that is able '
 'to lead, monitor, coordinate and report on efforts to combat corruption '
 'across the country. The project will work and implement a program of '
 'institutional development and sustainability. It will also support the '
 'office in carrying out its priority responsibilities in asset registration '
 'and verification, complaints management and case tracking, and coordination '
 'and monitoring of anticorruption performance across other government '
 'agencies.']


In [10]:
#tokenize and clean up using gensim 

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])


[['the', 'five', 'components', 'of', 'this', 'program', 'seek', 'to', 'take', 'what', 'already', 'exists', 'and', 'improve', 'upon', 'it', 'implement', 'it', 'and', 'find', 'ways', 'to', 'ensure', 'that', 'the', 'progress', 'achieved', 'is', 'sustainable', 'over', 'the', 'long', 'term', 'the', 'project', 'will', 'help', 'the', 'government', 'of', 'the', 'islamic', 'republic', 'of', 'afghanistan', 'strengthen', 'its', 'high', 'office', 'of', 'oversight', 'hoo', 'making', 'it', 'strong', 'effective', 'institution', 'that', 'is', 'able', 'to', 'lead', 'monitor', 'coordinate', 'and', 'report', 'on', 'efforts', 'to', 'combat', 'corruption', 'across', 'the', 'country', 'the', 'project', 'will', 'work', 'and', 'implement', 'program', 'of', 'institutional', 'development', 'and', 'sustainability', 'it', 'will', 'also', 'support', 'the', 'office', 'in', 'carrying', 'out', 'its', 'priority', 'in', 'asset', 'registration', 'and', 'verification', 'complaints', 'management', 'and', 'case', 'tracking

In [11]:
#lemmatization

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

['component program seek take already exist improve implement find way ensure progress achieve sustainable long term project will help government strengthen high office oversight hoo make strong effective institution able lead monitor coordinate report effort combat corruption country project will work implement program institutional development sustainability will also support office carry priority asset registration verification complaint management case tracking coordination monitoring anticorruption performance other government agency', 'overall goal program support rule law strategy specifically pillar improve justice sector reduce corruption pillar build leadership justice sector civil society support pillar contractor attempt develop capacity judiciary law school pillar contractor work raise public awareness encourage citizen resolve dispute formal justice sector program divide component capacity building judiciary capacity building court management system capacity building facu

In [12]:
## TFIDF


vectorizer = TfidfVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             ngram_range=(1,3)             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [13]:
#Checking the sparcity

# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  9.346462864981383 %


In [17]:
## BASE MODEL



# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=20,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(learning_method='online', n_components=20, n_jobs=-1,
                          random_state=100)


In [18]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -11455.377931215236
Perplexity:  1866.1054411160621
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 20,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [26]:
## HYPERPARAMETER TUNING

# Define Search Param
search_params = {'n_components': np.arange(5,30).tolist(), 'learning_decay': [.3,.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

GridSearchCV(estimator=LatentDirichletAllocation(),
             param_grid={'learning_decay': [0.3, 0.5, 0.7, 0.9],
                         'n_components': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,
                                          16, 17, 18, 19, 20, 21, 22, 23, 24,
                                          25, 26, 27, 28, 29]})

In [27]:
## BEST TOPIC MODEL


# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))



Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -2437.6452269627785
Model Perplexity:  775.3682121673747


In [29]:
### DOMINANT TOPIC IN EACH DOCUMENT


# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics




In [30]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
0      150.222153  -72.269554       1        1  59.102340
4       51.163364 -159.968018       2        1  29.627879
2      -73.144554   21.523815       3        1   4.112624
3      -66.410133 -110.606438       4        1   3.638186
1       54.417324   12.443444       5        1   3.518971, topic_info=            Term       Freq      Total Category  logprob  loglift
72    corruption  16.000000  16.000000  Default  30.0000  30.0000
25    assessment   5.000000   5.000000  Default  29.0000  29.0000
222    political   6.000000   6.000000  Default  28.0000  28.0000
169      justice   9.000000   9.000000  Default  27.0000  27.0000
231  procurement   5.000000   5.000000  Default  26.0000  26.0000
..           ...        ...        ...      ...      ...      ...
134       health   0.159416   6.192466   Topic5  -5.8163  -0.3126
38     budgeting   0.159410   2.757525   Topic5  -5.8164   0.4964
117       fiscal   0.159409   3.307455   Topic5  -5.8164   0.3145
106       engage   0.159408   3.047487   Topic5  -5.8164   0.3964
262       region   0.159401   4.184855   Topic5  -5.8164   0.0792

[283 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
2         1  0.774669  accountability
2         2  0.258223  accountability
9         1  0.197887  administration
9         2  0.593662  administration
10        1  0.397866  administrative
...     ...       ...             ...
321       1  0.915057           woman
322       1  0.722591            work
322       2  0.240864            work
323       1  0.365179            year
323       2  0.365179            year

[263 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 3, 4, 2])

In [32]:
### TOPIC KEYWORDS ###

# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames

# View
df_topic_keywords.head()

ability    access  accountability  accountable   achieve    action  \
Topic0  1.285317  3.139387        6.694171     2.601733  3.439739  1.044403   
Topic1  0.200173  0.200166        0.200150     0.200155  0.200181  0.200183   
Topic2  0.200149  0.200170        0.200213     0.200132  0.200226  0.200553   
Topic3  0.200151  0.200146        0.202567     0.200374  0.200416  0.200161   
Topic4  1.589258  3.030433        2.021745     1.883327  0.661555  1.511100   

        activity     actor   address  administration  ...  training  \
Topic0  3.410162  1.596314  3.260764        1.573688  ...  6.300626   
Topic1  0.200149  0.200201  0.200168        0.200167  ...  0.200162   
Topic2  0.200268  0.200803  0.200411        0.200238  ...  0.200264   
Topic3  0.200129  0.200175  0.200156        0.200695  ...  0.228670   
Topic4  3.546307  1.774689  2.135536        4.059813  ...  1.954948   

        transition  transparency  transparency accountability  transparent  \
Topic0    2.876376      6.906762                     4.505141     4.332309   
Topic1    0.200126      0.200186                     0.200168     0.200151   
Topic2    0.200108      0.467312                     0.200145     0.200140   
Topic3    0.200109      0.210951                     0.206162     0.200388   
Topic4    0.202061      3.555961                     0.400683     1.614267   

            unit       use     woman      work      year  
Topic0  1.770925  3.661332  4.424964  7.089657  1.277014  
Topic1  0.200212  0.200214  0.200122  0.200158  0.200264  
Topic2  0.200184  0.499216  0.201040  0.200295  0.564439  
Topic3  0.200184  0.277019  0.200683  0.200213  0.200645  
Topic4  0.578098  0.668087  0.201336  2.304167  1.121570  

[5 rows x 324 columns]

In [33]:
### TOP 15 KEYWORDS FOR EACH TOPIC ###

# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

Word 0               Word 1       Word 2  \
Topic 0                          local           government      project   
Topic 1                      political          stakeholder   assessment   
Topic 2                     assessment               report   corruption   
Topic 3  technical assistance training  assistance training  procurement   
Topic 4                     corruption              justice     judicial   

                     Word 3      Word 4       Word 5  \
Topic 0             program  governance      support   
Topic 1         association     conduct  information   
Topic 2              region        task         anti   
Topic 3  public procurement    approach        great   
Topic 4                 law       legal      project   

                               Word 6    Word 7                Word 8  \
Topic 0                       improve  capacity               service   
Topic 1                       program   network              regional   
Topic 2                     encourage   program                   non   
Topic 3  provide technical assistance       use  technical assistance   
Topic 4                         court      rule                reform   

                    Word 9       Word 10            Word 11         Word 12  \
Topic 0             public   development         strengthen       community   
Topic 1            prevent   enforcement  combat corruption          report   
Topic 2             ensure  governmental               year  anticorruption   
Topic 3  provide technical           new           training       technical   
Topic 4         strengthen         trade           rule law         program   

                Word 13                 Word 14  
Topic 0             aim        local government  
Topic 1  capacity civil  capacity civil society  
Topic 2             use               objective  
Topic 3  capacity civil  capacity civil society  
Topic 4        increase                 improve